In [1]:
import faiss
import pandas as pd
import numpy as np
import transformers
import json
import torch
import time
from tqdm.notebook import tqdm
import pickle

In [2]:
OUTFILE = <FILL OUT FILE NAME HERE>
df = pd.read_json(OUTFILE, lines=True)

In [3]:
MODEL = 'bert-base-uncased'

sciBert = transformers.BertModel.from_pretrained(MODEL)
tokenizer = transformers.BertTokenizer.from_pretrained(MODEL, do_lower_case=True)

device = "cuda:0"
sciBert = sciBert.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
CHUNK_SIZE_EACH = len(df)

def __embedding(text):
    return compute_mean(embedding_fn(sciBert, text))

def compute_bert_embeddings(dataframe_chunk, current_index, end_marker, filename):
    np_chunk = __embedding(dataframe_chunk.loc[current_index * end_marker]['text']).detach().numpy()

    for idx in range(1, end_marker):
        try:
            embedding = __embedding(dataframe_chunk.loc[(current_index * end_marker) + idx]['text']).detach().numpy()
            np_chunk = np.append(np_chunk, embedding, axis = 0)
            print('\r {}'.format(np_chunk.shape), end = '')
        except Exception as e:
            print(e)
            np_chunk = np.append(np_chunk, np.zeros(shape = (1, 768)), axis = 0)
            continue 

    print(np_chunk.shape)
    np.savez_compressed(filename.format(current_index), a = np_chunk)


def compute_embeddings_and_save(dataframe, filename):
    n_rows = len(dataframe)

    chunk_sizes = n_rows // CHUNK_SIZE_EACH
    remaining = n_rows - chunk_sizes * CHUNK_SIZE_EACH

    for i in range(1):
        compute_bert_embeddings(dataframe[i * CHUNK_SIZE_EACH : (i * CHUNK_SIZE_EACH) + CHUNK_SIZE_EACH ], i, 
                                CHUNK_SIZE_EACH,  filename)

In [6]:
def embedding_fn(model, text) :
    with torch.no_grad():
        tokens = tokenizer.encode(text, max_length=512)
        batch_tokens = np.expand_dims(tokens, axis = 0)
        batch_tokens = torch.tensor(batch_tokens).cuda()
        batch_tokens = batch_tokens.to(device)
        return model(batch_tokens)[0].cpu()

def compute_mean(embedding):
    if not isinstance(embedding, torch.Tensor):
        print('Embedding must be a torch.Tensor')
        return 
    return embedding.mean(1)

def compute_cosine_measure(x1, x2):
    return cosine_similarity(x1, x2)

def compute_distance(x1, x2):
    return compute_cosine_measure(x1.detach().numpy(), x2.detach().numpy())

In [17]:
def generate_mapping_index(dataframe):
    index_map = {}

    for index, row in dataframe.iterrows():
        author_username = row['user']['screen_name']
        author_userid = row['user']['id']
        
        index_map[index] = {
            "cord_uid" : row['id'],
            "author_user_name" : author_username,
            "text" : row['text'],
            "author_userid": author_userid
        }

    return index_map

In [22]:
compute_embeddings_and_save(df, <FILL EMBEDDING OUTFILENAME>)

 (101453, 768)(101453, 768)


In [21]:
index_map = generate_mapping_index(df_trimmed)
open(<FILL MAPPING OUTFILE NAME>, 'w').write(json.dumps(index_map))

32638284